# Identificação de hackers do sistema de uma empresa

## Descrição do problema

Uma empresa sofreu ataques hacker em seu sistema e está tentando identificar os hackers envolvidos. Ela já verificou que pelo menos 2 indivíduos participaram dos ataques e agora está investigando a participação de um terceiro. 

O objetivo desse projeto e investigar se 2 ou 3 hackers foram os responsáveis pelos ataques. A empresa possuí dados de cada sessão de hackeamento. As informações são as seguintes:

    - Tempo de conexão em minutos
    - Quantidade de dados transferidos durante a sessão 
    - Uso do Kali Linux (Sim ou Não)
    - Número de servidores corrompidos
    - Número de páginas acessadas ilegalmente
    - Localidade do ataque 
    - Velocidade de digitação.
    
Essa investigação pode ser realizada utilizando algorítmos de clusterização de Machine Learning. 

Uma outra informação crucial na resolução dessa investigação é que os hackers dividem o número de ataques de forma igual entre os participantes. Dessa forma se ocorreram 100 ataques e 2 indivíduos foram envolvidos deve-se ter 50 ataques de cada. Por outro lado, se 3 pessoas participaram cada indivíduo efetuou aproximadamente 33 ataques. 

Esse problema é solucionado abaixo utilizando-se o método K-means de clusterização no pyspark. 

## Carregamento dos dados e pré-processamento

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('clustering').getOrCreate()

In [5]:
# carrega os dados
data = spark.read.csv('hack_data.csv', header = True, inferSchema=True)

In [6]:
data.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [7]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



## Clusterização

In [10]:
from pyspark.ml.clustering import KMeans

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

A localidade não será considerada na análise pois os ataques usualmente utilizam de VPNs e portanto essa informação não é útil.

In [14]:
# agrupa as variáveis que serão utilizadas na análise
assembler = VectorAssembler(inputCols=['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used'
                            ,'Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed'], outputCol='features')

In [15]:
final_data = assembler.transform(data)

O método K-means é baseado no cálculo de distâncias no espaço das variáveis. Portanto, para utilizá-lo as variáveis devem estar todas em escalas comparáveis. 

Abaixo a normalização das variáveis é realizada utilizando o pacote StandardScaler do pyspark. 

In [19]:
from pyspark.ml.feature import StandardScaler

In [20]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

In [21]:
# normaliza os dados
scaler_model = scaler.fit(final_data)

In [22]:
final_data = scaler_model.transform(final_data)

Para se identificar se 2 ou 3 hackers atuaram nos ataques a clusterização é realizada com 2 e 3 clusters.

In [23]:
kmeans2 = KMeans(featuresCol='scaled_features', k = 2)
kmeans3 = KMeans(featuresCol='scaled_features', k = 3)

In [25]:
model2 = kmeans2.fit(final_data)
model3 = kmeans3.fit(final_data)

In [27]:
predictions2 = model2.transform(final_data)
predictions3 = model3.transform(final_data)

In [35]:
predictions2.select('prediction').show(5)
predictions3.select('prediction').show(5)

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 5 rows

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
+----------+
only showing top 5 rows



In [26]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [42]:
# calcula o coeficiente de silhueta nos dois casos
evaluator = ClusteringEvaluator()
silhouette2 = evaluator.evaluate(predictions2)
silhouette3 = evaluator.evaluate(predictions3)

In [54]:
print('A coeficiente de silhueta quando tem-se 2 clusters é igual a {:.2f}'.format(silhouette2))
print('A coeficiente de silhueta quando tem-se 3 clusters é igual a {:.2f}'.format(silhouette3))


A coeficiente de silhueta quando tem-se 2 clusters é igual a 0.67
A coeficiente de silhueta quando tem-se 3 clusters é igual a 0.30


Os valores do coeficiente de silhueta já indicam que os ataques são melhor agrupados em 2 clusters e, portanto, apenas 2 indivíduos estariam envolvidos nos ataques.

Para confirmar esse resultado podemos verificar o número de ataques nos 2 casos, pois sabe-se que os hackers dividem igualmente os ataques. 

In [55]:
predictions2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [56]:
predictions3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   79|
|         2|  167|
|         0|   88|
+----------+-----+



Quando os ataques são dividos em 3 clusters o número de ataques associados a cada cluster é muito desigual.

Com 2 clusters um mesmo número de ataques é associado para cada. Dessa forma é bastante provável que apenas 2 hackers participaram dos ataques.